First we instantiate the simulator class we will use to generate our sequences.

In [1]:
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.detection import Detection

from stonesoup.types.track import Track
from stonesoup.types.hypothesis import SingleHypothesis

class simulator():

    def __init__(self, transition_model, measurement_model, state_range, meas_range = None, noise_range = None, meas_noise_range = None):
        """
        Parameters
        ----------
        transition_model : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        measurement_model : :class:`~.Predictor`
            The Updater to be used.
        """
        self.transition_model= transition_model
        self.measurement_model = measurement_model
        self.state_range = state_range
        self.meas_range = meas_range
        self.noise_range = noise_range
        self.meas_noise_range = meas_noise_range

    def _generate_ground_truth(self, prior, time_span):
        
        time_interval = time_span[1]-time_span[0]
        ground_truth = GroundTruthPath([prior])
        for k in range(1, len(time_span)):
            ground_truth.append(GroundTruthState(
                self.transition_model.function(ground_truth[k-1], noise=True, time_interval=time_interval),
                timestamp=time_span[k-1]))
        return ground_truth
    
    def _simulate_measurements(self, ground_truth):
        #Simulate Measurements
        measurements = []
        for state in ground_truth:
            measurement = self.measurement_model.function(state, noise=True)
            measurements.append(Detection(measurement,
                                          timestamp=state.timestamp,
                                          measurement_model=self.measurement_model))
        return measurements

    def _initializer(self, time_stamp):
        state_range_min = self.state_range[0]
        state_range_max = self.state_range[1]
        state_vector = StateVector(np.random.uniform(low=state_range_min, high=state_range_max))
        while np.sqrt(state_vector[1]**2+state_vector[3]**2) < 0.25:
            state_vector = StateVector(np.random.uniform(low=state_range_min, high=state_range_max))
        meas_range_low = self.meas_range[0]
        meas_range_max = self.meas_range[1]
        self.measurement_model.translation_offset = np.random.uniform(low=meas_range_low, high=meas_range_max).reshape(len(meas_range_low),1)
        #process_noise = 
        #self.transition_model. = np.randint(low=0,high=len(noise_range))
        
        return State(state_vector = state_vector, timestamp=time_stamp)

    def generate_training_data(self, time_span):
        """

        Parameters
        ----------
        ground_truth : :class:`~.GroundTruthPath`
            StateMutableSequence type object used to store ground truth.
        initial_state : :class:`~.State`
            Initial state for the ground truth system. This MUST be a State,
            not a State subclass, like GaussianState or EnsembleState.
        prior : :class:`~.GaussianState` or :class:`~.EnsembleState`
            Initial state prediction of tracking algorithm.

        Returns
        -------
        track : :class:`~.Track`
            The Stone Soup track object which contains the list of updated 
            state predictions made by the tracking algorithm employed.
        """

        #Simulate Measurements
        initial_state = self._initializer(time_span[0])
        ground_truth = self._generate_ground_truth(initial_state, time_span)
        measurements = self._simulate_measurements(ground_truth)
        
        return ground_truth, measurements

    def simulate_track(self, predictor, updater, initial_state, prior, time_span):
        """

        Parameters
        ----------
        predictor : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        updater : :class:`~.Predictor`
            The Updater to be used.
        ground_truth : :class:`~.GroundTruthPath`
            StateMutableSequence type object used to store ground truth.
        initial_state : :class:`~.State`
            Initial state for the ground truth system. This MUST be a State,
            not a State subclass, like GaussianState or EnsembleState.
        prior : :class:`~.GaussianState` or :class:`~.EnsembleState`
            Initial state prediction of tracking algorithm.

        Returns
        -------
        track : :class:`~.Track`
            The Stone Soup track object which contains the list of updated 
            state predictions made by the tracking algorithm employed.
        """

        #Simulate Measurements
        ground_truth = self._generate_ground_truth(initial_state, time_span)
        measurements = self._simulate_measurements(ground_truth)
        
        #Initialize Loop Variables
        track = Track()
        for measurement in measurements:
            prediction = predictor.predict(prior, timestamp=measurement.timestamp)
            hypothesis = SingleHypothesis(prediction, measurement)  # Group a prediction and measurement
            posterior = updater.update(hypothesis)
            track.append(posterior)
            prior = track[-1]
        return ground_truth, track

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import datetime

from stonesoup.types.array import StateVector, CovarianceMatrix
from stonesoup.types.state import State, GaussianState

from stonesoup.models.transition.linear import (CombinedLinearGaussianTransitionModel,
                                                ConstantVelocity)
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.updater.kalman import KalmanUpdater, ExtendedKalmanUpdater
from stonesoup.predictor.ensemble import EnsemblePredictor
from stonesoup.predictor.kalman import KalmanPredictor, ExtendedKalmanPredictor
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange

"""
    This script represents the code used to gather the data used in [PAPER HERE].
    
    This repository is structured such that different stone soup algorithms 
    can be run rapidly. Hopefully I've made it modular enough to 
    allow swapping of things like algorithms, and "experiments" by replacing
    the desired transition and measurement models.
    
    The simulator class requires a transition and 
    measurement model, then the simulate_track method accepts a Stone Soup
    Predictor, Updater, ground truth initial state, initial state for the
    chosen algorithm, and a span of time which the simulation takes place over.
    This time span should be an evenly spaced datetime.datetime list.
    
    The simulator then, is used to gather "Track" instances, and with a list 
    of tracks, RMSE can then be calculated.
"""

i = 60
num_vectors = i*5

"""
    Here, we get our initial variables for simulation. For this, we are just
    using a time span of 60 time instances spaced one second apart.
"""

timestamp = datetime.datetime(2021, 4, 2, 12, 28, 57)
tMax = 60
dt = 1
tRange = tMax // dt
plot_time_span = np.array([dt*i for i in range(tRange)])

time_span = np.array([timestamp + datetime.timedelta(seconds=dt*i) for i in range(tRange)])

"""
Here we instantiate our transition and measurement models. These are the 
same models used in the StoneSoup Kalman Filter examples.
"""

q_x = 0.05
q_y = 0.05
sensor_x = 0  # Placing the sensor off-centre
sensor_y = 0

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

measurement_model = CartesianToBearingRange(
ndim_state=4,
mapping=(0, 2),
noise_covar=np.diag([np.radians(1), 1]),  # Covariance matrix. 0.2 degree variance in
# bearing and 1 metre in range
translation_offset=np.array([[sensor_x], [sensor_y]])  # Offset measurements to location of
# sensor in cartesian.
)


Now with our simulator we need to instantiate the Pytorch Dataset.

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy as np

# Datasets should be specified on a per simulation basis

#class sequence_generator(Dataset):

class dataset_2D_bearing_range(Dataset):
    #Range Bearing 2D Dataset Sequence Packer

    def __init__(self, dataset, ground_truth=None, measurements=None):
        if dataset == None:   
            gt = np.array([e.state_vector for e in ground_truth]).squeeze().T
            gt = torch.tensor(gt.astype(dtype=np.float32),device = device)
            ms = np.array([m.state_vector for m in measurements]).squeeze().T
            ms = torch.tensor(ms.astype(dtype=np.float32),device = device)
            self.dataset = torch.unsqueeze(torch.cat((ms,gt)),dim=0)
        else:
            self.dataset = dataset
            
    
    def append_to_dataset(self, ground_truth, measurements):
        """
        Parameters
        ----------
        ground_truth : :class:`~.list`
            A list of Stonesoup States.
        measurements : :class:`~.measurements`
            The list of Stonesoup Measurements to be used.
        """
    
        gt = np.array([e.state_vector for e in ground_truth]).squeeze().T
        gt = torch.tensor(gt.astype(dtype=np.float32),device = device)
        ms = np.array([m.state_vector for m in measurements]).squeeze().T
        ms = torch.tensor(ms.astype(dtype=np.float32),device = device)
        new_entry = torch.cat((ms,gt),dim=0)

        self.dataset = torch.cat((self.dataset,torch.unsqueeze(new_entry,dim=0)),dim=0)

    def __getitem__(self, idx):
        return self.dataset[idx,0:2].T, self.dataset[idx,2:6].T

    def __len__(self):
        return self.dataset.shape[0]



cuda


Now we need to simulate and add to our dataset.

In [4]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy as np

# Datasets should be specified on a per simulation basis
# Should look up realistic values for radar precision and initial states
# Come up with realistic constraints for initial velocities

#class sequence_generator(Dataset):

class dataset_composer():
    #Range Bearing 2D Dataset Sequence Packer
    def __init__(self, simulators, training_dataset, testing_dataset):
        #pass a list of simulators
        self.simulators = simulators
        self.training_dataset = training_dataset
        self.testing_dataset = testing_dataset

    def simulate_trajectories(self, training_dataset_len, testing_ratio):
        for simulator in self.simulators:
            trainset_length = int((training_dataset_len-1) / len(self.simulators)) # Partition dataset into sets from each simulator
            for i in range(trainset_length):
                ground_truth, measurements = simulator.generate_training_data(time_span)
                self.training_dataset.append_to_dataset(ground_truth, measurements)
            trainloader = DataLoader(self.training_dataset)

            testset_length = int(((training_dataset_len)*testing_ratio -1)/len(self.simulators))
            for i in range(testset_length):
                ground_truth, measurements = simulator.generate_training_data(time_span)
                self.testing_dataset.append_to_dataset(ground_truth, measurements)
            testloader = DataLoader(self.testing_dataset)
        return trainloader, testloader

    def output_to_file(self, trainset_name, testset_name):
        torch.save(self.training_dataset, trainset_name)
        torch.save(self.testing_dataset, testset_name)

    def read_from_file(self, training_file_name, testing_file_name, device):
        trn = torch.load(training_file_name, map_location = device)
        tst = torch.load(testing_file_name, map_location = device)
        return DataLoader(trn), DataLoader(tst)




In [5]:
# Position, Velocity, Position, Velocity
state_range_min = np.array([-25, -2, -25, -2])
state_range_max = np.array([25, 2, 25, 2])

# Add some sort of miniumum speed

meas_range_min = np.array([0, 0])
meas_range_max = np.array([0, 0])

# Randomization for process noise
process_noise_coefficients = [0.005, 0.05, 0.5]

nonlinear_simulator = simulator(transition_model=transition_model,
                      measurement_model=measurement_model,
                      state_range = (state_range_min, state_range_max),
                      meas_range = (meas_range_min, meas_range_max)
)

ground_truth, measurements = nonlinear_simulator.generate_training_data(time_span)
training_dataset =  dataset_2D_bearing_range(dataset = None, ground_truth = ground_truth, measurements = measurements)
testing_dataset =  dataset_2D_bearing_range(dataset = None, ground_truth = ground_truth, measurements = measurements)

#training_dataset_len = 1000000 #  1,000,000
training_dataset_len = 100000 #  100,000
#training_dataset_len = 10000
testing_ratio = 0.25

dataset_composer = dataset_composer([nonlinear_simulator], testing_dataset, training_dataset)

#trainloader, testloader = dataset_composer.simulate_trajectories(training_dataset_len, testing_ratio)
#dataset_composer.output_to_file('training_data','testing_data')
trainloader, testloader = dataset_composer.read_from_file('training_data', 'testing_data', device)
#print('Training Dataset Length:', training_dataset.__len__())
#print('Testing Dataset Length:',testing_dataset.__len__())


Turn Sequences into dataset

In [6]:
import torch.nn as nn
import torch.nn.functional as F

input_size = 2
hidden_size = 32
num_layers = 1
nonlinearity = 'relu'
output_size = 4

class BiDirectionalRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, nonlinearity, num_layers, device):
        super(BiDirectionalRNN, self).__init__()
        self.RNN = torch.nn.RNN(input_size, hidden_size, bidirectional=True, num_layers=num_layers, device=device)
        self.linear = torch.nn.Linear(2*hidden_size, output_size, device=device)

    def forward(self, input, RNN_hidden):
        RNN_output, RNN_hidden = self.RNN(input, RNN_hidden)
        output = self.linear(RNN_output)
        return output, RNN_hidden

model = BiDirectionalRNN(input_size, hidden_size, output_size, nonlinearity, num_layers=num_layers, device=device)
#model = EncoderDecoderRNN(input_size, hidden_size, output_size, num_layers=num_layers, device=device)

In [7]:
criterion = nn.MSELoss()
learning_rate = 0.00005;
weight_decay = 0.005 #L2 Regularization Factor
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate, weight_decay = weight_decay)

In [ ]:
n_epochs = 50
RNN_hidden = torch.zeros((2, tMax, hidden_size),device=device)

training_loss = torch.zeros(len(trainloader))
testing_loss = torch.zeros(len(testloader))
training_loss_epoch = torch.zeros(n_epochs)
testing_loss_epoch = torch.zeros(n_epochs)
testing_MSE = torch.zeros(1, tMax, output_size, device=device)
for i in range(n_epochs):
    for j, data in enumerate(trainloader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs, encoder_hidden = model(inputs, RNN_hidden.detach())
        train_loss = criterion(outputs, labels)
        training_loss[j] = train_loss
        train_loss.backward()
        optimizer.step()
    training_loss_epoch[i] = torch.mean(train_loss).item()
    with (torch.no_grad()):
        for j, data in enumerate(testloader):
            inputs, labels = data
            outputs, encoder_hidden = model(inputs, RNN_hidden.detach())
            test_loss = criterion(outputs, labels)
            testing_loss[j] = test_loss
        testing_loss_epoch[i] = torch.mean(testing_loss).item()

    print("Epoch:",i+1, "\nTraining Loss:", training_loss_epoch[i].item(),
          "\nTesting Loss:", testing_loss_epoch[i].item(),
          "\n---------------------------")

Epoch: 1 
Training Loss: 938.8143310546875 
Testing Loss: 750.7586669921875 
---------------------------
Epoch: 2 
Training Loss: 560.3429565429688 
Testing Loss: 563.3450927734375 
---------------------------


Plotting

Trajectory MSE Plot

In [ ]:
trajectory_MSE = torch.zeros(tMax, output_size, device=device)
trajectory_mse_func = nn.MSELoss(reduction='none')
with (torch.no_grad()):
    j = 0
    for j, data in enumerate(testloader):
        inputs, labels = data
        outputs, encoder_hidden = model(inputs, RNN_hidden.detach())
        trajectory_MSE += trajectory_mse_func(outputs.squeeze(), labels.squeeze())
        j = j+1
    testing_MSE = trajectory_MSE / j

import matplotlib.pyplot as plt

fig, RNN_combined_plot = plt.subplots(1,2)

RNN_x = testing_MSE.T[0].cpu().numpy()
RNN_xv = testing_MSE.T[1].cpu().numpy()
RNN_y = testing_MSE.T[2].cpu().numpy()
RNN_yv = testing_MSE.T[3].cpu().numpy()
t = range(len(time_span))

RNN_pos = np.sqrt( (RNN_x + RNN_y) / 2 )
RNN_vel = np.sqrt( (RNN_xv + RNN_yv) / 2 )

t = range(len(time_span))
height = 3

width = 9

fig.tight_layout()
RNN_combined_plot[0].plot(t, RNN_pos, color="C0")
RNN_combined_plot[0].set_xlabel("Time (s)")
RNN_combined_plot[0].set_ylabel("Root Mean Square Error (km)")
RNN_combined_plot[0].set_title("Avg RMSE vs Time")
RNN_combined_plot[0].figure.set_figwidth(width) # 6.4
RNN_combined_plot[0].figure.set_figheight(height) # 6.4
RNN_combined_plot[1].plot(t, RNN_vel, color="C1")
RNN_combined_plot[1].set_xlabel("Time (s)")
RNN_combined_plot[1].set_ylabel("Root Mean Square Error (km/s)")
RNN_combined_plot[1].set_title("Avg RMSE vs Time")
RNN_combined_plot[1].figure.set_figwidth(width) # 6.4
RNN_combined_plot[1].figure.set_figheight(height) # 6.4

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2,1)

x = range(n_epochs)

ax[0].set_title("Training Loss (MSE)")
ax[0].plot(x, training_loss_epoch,color="C0")
fig.tight_layout()
ax[1].set_title("Testing Loss (MSE)")
ax[1].plot(x, testing_loss_epoch,color="C1")

Track Simulator

In [ ]:
class track_simulator():
    def __init__(self, predictor, updater, dataloader, time_span):
        """
        Parameters
        ----------
        transition_model : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        measurement_model : :class:`~.Predictor`
            The Updater to be used.
        """
        self.predictor = predictor
        self.updater = updater
        self.dataloader = dataloader
        self.time_span = time_span

    def initializer(self, time_stamp):
        state_range_min = np.array([-25, -2, -25, -2])
        state_range_max = np.array([25, 2, 25, 2])
        state_vector = StateVector(np.random.uniform(low=state_range_min, high=state_range_max))
        return GaussianState(state_vector = state_vector, covar = np.eye(4), timestamp=time_stamp)

    def _simulate_track(self, input):
        track = Track()
        prior = self.initializer(self.time_span[0])
        for i in range(inputs.shape[1]):
            measurement = Detection(state_vector=input[0,i].cpu().numpy(), timestamp = self.time_span[i])
            prediction = self.predictor.predict(prior, timestamp=measurement.timestamp)
            posterior = self.updater.update(SingleHypothesis(prediction, measurement))
            track.append(posterior)
            prior = track[-1]
        tk = np.array([e.state_vector for e in track]).squeeze().T
        tk = torch.tensor(tk.astype(dtype=np.float32),device = device)
        return tk.T

    def generate_MSE(self):
        avg_loss = nn.MSELoss()
        trajectory_MSE = nn.MSELoss(reduction='none')
        MSE = torch.zeros(1,60,4, device=device) # Make this auto detect it
        for j, data in enumerate(self.dataloader):
            inputs, labels = data
            outputs = self._simulate_track(inputs)
            test_loss = avg_loss(outputs, labels.squeeze())
            MSE += trajectory_MSE(outputs, labels.squeeze())
            testing_loss[j] = test_loss
        return torch.mean(testing_loss).numpy(), MSE.squeeze()/len(self.dataloader)

In [ ]:
predictor = ExtendedKalmanPredictor(transition_model)
updater = ExtendedKalmanUpdater(measurement_model)

track_simulator = track_simulator(predictor, updater, testloader, time_span)

EKF_epoch_loss, EKF_track = track_simulator.generate_MSE()

track_simulator.generate_MSE()

epoch_mse, EKF_MSE = track_simulator.generate_MSE()


Plotting

In [ ]:
import matplotlib.pyplot as plt
fig, EKF_mse_plot = plt.subplots(2,2)

EKF_x = EKF_MSE.T[0].cpu().numpy()
EKF_xv = EKF_MSE.T[1].cpu().numpy()
EKF_y = EKF_MSE.T[2].cpu().numpy()
EKF_yv = EKF_MSE.T[3].cpu().numpy()
t = range(len(time_span))

fig.tight_layout()
EKF_mse_plot[0][0].plot(t, EKF_x, color="C0")
EKF_mse_plot[0][1].plot(t, EKF_xv, color="C1")
EKF_mse_plot[1][0].plot(t, EKF_y, color="C2")
EKF_mse_plot[1][1].plot(t, EKF_yv, color="C3")
#EKFmse[1].set_title("Testing Loss (MSE)")


In [ ]:
import matplotlib.pyplot as plt

fig, EKF_combined_plot = plt.subplots(1,2)

EKF_x = EKF_MSE.T[0].cpu().numpy()
EKF_xv = EKF_MSE.T[1].cpu().numpy()
EKF_y = EKF_MSE.T[2].cpu().numpy()
EKF_yv = EKF_MSE.T[3].cpu().numpy()
t = range(len(time_span))

EKF_pos = np.sqrt( (EKF_x + EKF_y) / 2 )
EKF_vel = np.sqrt( (EKF_xv + EKF_yv) / 2 )

t = range(len(time_span))
height = 3


width = 9

fig.tight_layout()
EKF_combined_plot[0].plot(t, EKF_pos, color="C0")
EKF_combined_plot[0].set_xlabel("Time (s)")
EKF_combined_plot[0].set_ylabel("Root Mean Square Error (km)")
EKF_combined_plot[0].set_title("Avg RMSE vs Time")
EKF_combined_plot[0].figure.set_figwidth(width) # 6.4
EKF_combined_plot[0].figure.set_figheight(height) # 6.4
EKF_combined_plot[1].plot(t, EKF_vel, color="C1")
EKF_combined_plot[1].set_xlabel("Time (s)")
EKF_combined_plot[1].set_ylabel("Root Mean Square Error (km/s)")
EKF_combined_plot[1].set_title("Avg RMSE vs Time")
EKF_combined_plot[1].figure.set_figwidth(width) # 6.4
EKF_combined_plot[1].figure.set_figheight(height) # 6.4